In [18]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import datetime
import pandas as pd
import numpy as np
from dashgrid import dgrid_components as dgc
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)


In [19]:
all_days=1000
end_date = datetime.datetime.now()
beg_date = end_date - datetime.timedelta(all_days)
date_series = pd.bdate_range(beg_date,end_date)
trade_dates = date_series.astype(str).str.replace('-','').astype(int)
n = len(trade_dates)
changes = np.random.lognormal(0,.15/256**.5,n-1)
initial = np.array([100.0])
closes = np.cumprod(np.append(initial,changes)).round(2)
open_ranges = np.random.lognormal(0,.3/256**.5,n)
opens = (closes * open_ranges).round(2)
low_ranges = np.random.lognormal(.1,.2/256**.5,n)
lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
lows = lows.round(2)
high_ranges = np.random.lognormal(.1,.2/256**.5,n)
highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
highs = highs.round(2)

volume_changes = np.random.lognormal(0,.50/256**.5,n-1)
initial_volume = np.array([1000000.0]) * np.random.lognormal(0,.15/256**.5)
volumes = np.cumprod(np.append(initial_volume,volume_changes)).round(0)


df_pseudo = dgc.make_df({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                  'close':closes,'volume':volumes})


In [20]:
fig1 = dgc.PlotlyCandles(df_pseudo.iloc[-50:],number_of_ticks_display=20,title='Random Walk')
iplot(fig1.get_figure())

In [21]:
no_border={
    'border': '1px solid #000',
    'grid-gap': '10px 10px',
}

logger = dgc.li.init_root_logger('logfile.log','INFO')

def dash_app_1():
    days_to_show = 50
    df_to_show = df_pseudo.iloc[-days_to_show:]
    
    dt1_comp = dgc.DashTableComponent(
        'dt1',df_to_show,None,title='Random Walk',
        editable_columns=['close'],logger = logger
    )
    def create_figure_from_df(input_list):
        if input_list is None or len(input_list)<1 or input_list[0] is None:
            return [None]
        dict_df = input_list[0]
        df = dgc.make_df(dict_df)
        df.close = df.close.astype(str).astype(float)
        p = dgc.PlotlyCandles(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        return p
    
    gr1_comp = dgc.FigureComponent('f1',dt1_comp,create_figure_from_df,logger=logger)

    app = dash.Dash()
    comp_list = [dt1_comp,gr1_comp]
    html_list = [dgc.create_grid([c],num_columns=1) for c in comp_list]
    app.layout = dgc.html.Div(html_list)
    [c.callback(app) for c in comp_list]
    return app


In [22]:
DASH_APP_TO_RUN = dash_app_1
app = DASH_APP_TO_RUN()
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2019-07-26 09:03:31,820 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)


In [23]:
l = dgc.flatten_layout(app)
for c in l:
    print('***********************************************')
    print(c)

***********************************************
Loading(children=Div([Div(children=[Div([Div(children=Div(children='Random Walk', style={'display': 'table-cell', 'textAlign': 'center', 'vertical-align': 'middle', 'font-size': '16px', 'font-weight': 'bold', 'width': '100%'}), style={'display': 'table', 'width': '100%'}), DataTable(columns=[{'name': 'date', 'id': 'date', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'close', 'id': 'close', 'editable': True}, {'name': 'volume', 'id': 'volume', 'editable': False}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'date': 20190520.0, 'open': 113.17, 'high': 116.38, 'low': 112.08, 'close': 115.27, 'volume': 621296.0}, {'date': 20190521.0, 'open': 116.65, 'high': 117.73, 'low': 115.22, 'close': 116.3, 'volume': 637415.0}, {'date': 20190522.0, 'open': 119.77, 'high': 120.86, 'low': 115.81, 'close': 11

In [24]:

dgc.dataframe_rounder(df_pseudo.head(),columns_to_round=['open'],digits=1)

,date,open,high,low,close,volume
0,20161031,100.1,101.16,98.93,100.00,1003196.0
1,20161101,99.6,101.75,98.47,100.64,968376.0
2,20161102,101.2,102.34,98.06,99.16,963503.0
3,20161103,100.5,101.59,98.51,99.60,962542.0
4,20161104,99.6,100.75,96.79,97.90,912761.0


In [26]:
df1 = df_pseudo.head()

In [27]:
df1.to_dict('rows')

[{'date': 20161031.0,
  'open': 100.06,
  'high': 101.16,
  'low': 98.93,
  'close': 100.0,
  'volume': 1003196.0},
 {'date': 20161101.0,
  'open': 99.57,
  'high': 101.75,
  'low': 98.47,
  'close': 100.64,
  'volume': 968376.0},
 {'date': 20161102.0,
  'open': 101.23,
  'high': 102.34,
  'low': 98.06,
  'close': 99.16,
  'volume': 963503.0},
 {'date': 20161103.0,
  'open': 100.49,
  'high': 101.59,
  'low': 98.51,
  'close': 99.6,
  'volume': 962542.0},
 {'date': 20161104.0,
  'open': 99.63,
  'high': 100.75,
  'low': 96.79,
  'close': 97.9,
  'volume': 912761.0}]

In [28]:
df1.to_dict('records')

[{'date': 20161031.0,
  'open': 100.06,
  'high': 101.16,
  'low': 98.93,
  'close': 100.0,
  'volume': 1003196.0},
 {'date': 20161101.0,
  'open': 99.57,
  'high': 101.75,
  'low': 98.47,
  'close': 100.64,
  'volume': 968376.0},
 {'date': 20161102.0,
  'open': 101.23,
  'high': 102.34,
  'low': 98.06,
  'close': 99.16,
  'volume': 963503.0},
 {'date': 20161103.0,
  'open': 100.49,
  'high': 101.59,
  'low': 98.51,
  'close': 99.6,
  'volume': 962542.0},
 {'date': 20161104.0,
  'open': 99.63,
  'high': 100.75,
  'low': 96.79,
  'close': 97.9,
  'volume': 912761.0}]

In [29]:
df1.to_dict()

{'date': {0: 20161031, 1: 20161101, 2: 20161102, 3: 20161103, 4: 20161104},
 'open': {0: 100.06, 1: 99.57, 2: 101.23, 3: 100.49, 4: 99.63},
 'high': {0: 101.16, 1: 101.75, 2: 102.34, 3: 101.59, 4: 100.75},
 'low': {0: 98.93, 1: 98.47, 2: 98.06, 3: 98.51, 4: 96.79},
 'close': {0: 100.0, 1: 100.64, 2: 99.16, 3: 99.6, 4: 97.9},
 'volume': {0: 1003196.0, 1: 968376.0, 2: 963503.0, 3: 962542.0, 4: 912761.0}}